In [0]:
%sh ls -l

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
dbutils.widgets.text("config_file", "dev-config.json")

In [0]:
config_file = dbutils.widgets.get("config_file")

In [0]:
import json

with open(config_file) as file:
    config = json.load(file)
print(config)

In [0]:
import pandas as pd

if config["env"] == "dev":
  df = pd.read_csv("sample-data.csv")
elif config["env"] == "prod":
  df = pd.read_csv("/Volumes/path/to/data")
else:
  raise Exception("Invalid environment")

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

from mylib import sumRow

sum_rows_udf = udf(sumRow, IntegerType())

spark_df = spark.createDataFrame(df)
columns = spark_df.columns
transformed_df = spark_df.withColumn("sum", sumRow(*columns))

In [0]:
table_name = "my_demo_git_table_dev" if config["env"] == "dev" else "my_demo_git_table_prod"
transformed_df.write.option("overwriteSchema", "true").mode("overwrite").saveAsTable(table_name)